In [30]:
## Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim  as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
from torchvision.transforms import transforms

In [31]:
## Create Fully connected network

class NN(nn.Module):
    
    def __init__(self,input_shape,output_shape):
        super(NN,self).__init__()
        self.fc1 = nn.Linear(input_shape,50)
        self.fc2 = nn.Linear(50,output_shape)
        
    def forward(self,x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [32]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'mps')

In [33]:
# Hyperparameters
input_shape = 28*28
output_shape = 10
batch = 100
num_epoch = 10
learning_rate = 0.01

In [34]:
## load Dataset
train_dataset = datasets.MNIST(root = 'datasets/',train=True,download=True,transform = transforms.ToTensor())
train_loader = DataLoader(dataset=train_dataset,batch_size=batch,shuffle=True)

test_dataset = datasets.MNIST(root = 'datasets/',train=False,download=True,transform = transforms.ToTensor())
test_loader = DataLoader(dataset=test_dataset,batch_size=batch,shuffle=True)

In [35]:
## Initialize model 
model = NN(input_shape,output_shape).to(device)

In [36]:
## Losss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)

In [37]:
data, target = next(iter(train_loader))

In [38]:
## Train Network 
for epoch in range(num_epoch):
    losses = []
    
    for batch_idx, (data,target) in enumerate(train_loader):
        # Get data to cuda 
        data = data.to(device=device)
        target =target.to(device=device)
        
        # reshape
        data = data.view(data.shape[0],-1)
        
        # forward
        scores = model(data)
        loss = criterion(scores,target)
        losses.append(loss.item())
        
        # backward
        optimizer.zero_grad()
        loss.backward()
        
        # gradient descent
        optimizer.step()
        
    print(f"loss {sum(losses)/len(losses)}")

loss 0.2520673691481352
loss 0.12934234155652424
loss 0.10494690629187971
loss 0.0963946572287629
loss 0.08224051688720162
loss 0.08111215267543836
loss 0.07767363479399743
loss 0.07332593056179272
loss 0.06315640505267463
loss 0.06780451848860441


In [39]:
# check the accuracy of out trained model 
def check_accuracy(loader,model):
    for data,target in loader:
        num_correct = 0
        num_sample = 0
        model.eval()
        with torch.no_grad():
            data = data.to(device=device)
            target = target.to(device=device)

            # reshape 
            data = data.view(data.shape[0],-1)


            scores = model(data)
            _, pred = scores.max(1)
            # print(list(zip(pred,target)))
            num_correct += sum(pred == target)
            num_sample  += pred.shape[0]
    print(f'Total {num_correct} correct  / out of {num_sample} - accuracy {num_correct/num_sample :.3f} ')

  

In [40]:
# on test dataset
check_accuracy(test_loader,model)

Total 97 correct  / out of 100 - accuracy 0.970 


In [41]:
# on train datasets
check_accuracy(train_loader,model)

Total 100 correct  / out of 100 - accuracy 1.000 
